# Imports

In [4]:
import requests
import zipfile
import io
import sys

from lonboard import Map, SolidPolygonLayer
from lonboard.layer_extension import DataFilterExtension
import ipywidgets as widgets

import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.ticker as mticker

from shapely.geometry import Polygon

import seaborn as sns

# Fetch Data
Our population data is stored in a .csv file inside a zip folder. Let’s fetch it and load it into a pandas DataFrame.

In [5]:
url = "https://www.zensus2022.de/static/Zensus_Veroeffentlichung/Zensus2022_Bevoelkerungszahl.zip"
target_file = "Zensus2022_Bevoelkerungszahl_100m-Gitter.csv"

In [6]:
response = requests.get(url)

with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref: # Open the ZIP file in memory
    with zip_ref.open(target_file) as csv_file:
        df_pop = pd.read_csv(csv_file, delimiter=';') 